# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd
import numpy as np

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_toronto_postal = pd.read_html(url)[0]

# Preprocessing
# Clean Borough
df_toronto_postal_filter = df_toronto_postal[df_toronto_postal['Borough'] != 'Not assigned'].reset_index(drop=True)
# Merge Neighbourhood
df_toronto_postal_filter = df_toronto_postal_filter.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
# Clean Neighbourhood
mask = df_toronto_postal_filter['Neighbourhood'] == "Not assigned"
df_toronto_postal_filter.loc[mask, 'Neighbourhood'] = df_toronto_postal_filter.loc[mask, 'Borough']
print(f'The numbers of rows in the dataset are: {df_toronto_postal_filter.shape[0]}')

## Merge Geo location
# Read Data
geo = pd.read_csv('http://cocl.us/Geospatial_data')
# Merge Data
toronto_df_temp = df_toronto_postal_filter.set_index('Postal Code')
geo_temp = geo.set_index('Postal Code')
df_toronto_postal_filter = pd.concat([toronto_df_temp, geo_temp], axis=1, join='inner')
df_toronto_postal_filter.index.name = 'Postal Code'
df_toronto_postal_filter.reset_index(inplace=True)

print(df_toronto_postal_filter.shape)
df_toronto_postal_filter.head()

The numbers of rows in the dataset are: 103
(103, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Data Analysis

I'll focus in the toronto downtown for my analysis

In [2]:
# !conda install -c conda-forge folium=0.5.0 --yes
# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import folium
import requests

In [3]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [4]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
row_data = zip(
    df_toronto_postal_filter['Latitude'], 
    df_toronto_postal_filter['Longitude'], 
    df_toronto_postal_filter['Postal Code'], 
    df_toronto_postal_filter['Borough'], 
    df_toronto_postal_filter['Neighbourhood']
)

for lat, long, post, borough, neigh in row_data:
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [5]:
toronto_analysis = df_toronto_postal_filter[df_toronto_postal_filter['Borough'].str.contains('Downtown', na=False, case=False)]
print(toronto_analysis.shape)
toronto_analysis.head()

(19, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
51,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
53,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
54,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [6]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
row_data = zip(
    toronto_analysis['Latitude'], 
    toronto_analysis['Longitude'], 
    toronto_analysis['Postal Code'], 
    toronto_analysis['Borough'], 
    toronto_analysis['Neighbourhood']
)

for lat, long, post, borough, neigh in row_data:
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [78]:
# @hidden_cell
CLIENT_ID = 'Y4554RSKPQAK4CCMIER0M4YZSDN5CZRWO3ZQ23VTYGKNPVU3' # your Foursquare ID
CLIENT_SECRET = '23JRM4SFH1RJBOIA5JNZ24ZGCWVLZ3IDCUEO00SYEF15NELJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y4554RSKPQAK4CCMIER0M4YZSDN5CZRWO3ZQ23VTYGKNPVU3
CLIENT_SECRET:23JRM4SFH1RJBOIA5JNZ24ZGCWVLZ3IDCUEO00SYEF15NELJ


In [8]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list = []
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
toronto_downtown_venues = getNearbyVenues(
    names=toronto_analysis['Neighbourhood'],
    latitudes=toronto_analysis['Latitude'],
    longitudes=toronto_analysis['Longitude']
)

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [10]:
toronto_downtown_venues

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
...,...,...,...,...,...,...,...
1236,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,SUDS,43.659880,-79.394712,Bar
1237,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tim Hortons,43.658906,-79.388696,Coffee Shop
1238,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Understudy Café at Gerstein,43.662308,-79.394098,College Cafeteria
1239,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Arbor Room,43.663604,-79.394615,College Cafeteria


In [11]:
toronto_downtown_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,59,59,59,59,59,59
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,62,62,62,62,62,62
Christie,17,17,17,17,17,17
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [12]:
print('There are {} uniques categories.'.format(len(toronto_downtown_venues['Venue Category'].unique())))

There are 212 uniques categories.


In [13]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_downtown_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.00,0.00,0.016949,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.00,0.00,0.016129,0.000000,0.000000,0.016129,0.000000,0.00,0.016129
3,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Church and Wellesley,0.013158,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.013158,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.026316
5,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,...,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000
6,"First Canadian Place, Underground city",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.030000,0.000000,...,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000
7,"Garden District, Ryerson",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.01,0.00,0.00,0.000000,0.010000,0.010000,0.010000,0.000000,0.00,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.00,0.00,0.060606,0.000000,0.045455,0.015152,0.000000,0.00,0.000000


In [15]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----" + hood + "----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1      Farmers Market  0.03
2  Seafood Restaurant  0.03
3                Café  0.03
4         Cheese Shop  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Airport Lounge  0.12
1      Airport Service  0.12
2     Airport Terminal  0.12
3  Rental Car Location  0.06
4   Airport Food Court  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1                Café  0.05
2  Italian Restaurant  0.05
3      Sandwich Place  0.05
4        Burger Joint  0.03


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3          Diner  0.06
4    Candy Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.11
1  Japanese Restaurant  0.07
2     Sushi Restaurant  0.05
3           Restaura

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Farmers Market,Bakery,Café,Restaurant,Cocktail Bar,Seafood Restaurant,Beer Bar,Cheese Shop,Gym
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Bar,Rental Car Location,Sculpture Garden,Harbor / Marina,Boat or Ferry,Boutique
2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Japanese Restaurant,Burger Joint,Bubble Tea Shop,Department Store,Juice Bar
3,Christie,Grocery Store,Café,Park,Nightclub,Diner,Restaurant,Italian Restaurant,Baby Store,Candy Store,Athletics & Sports
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Café,Hotel,Mediterranean Restaurant,Yoga Studio,Men's Store


# Cluster Neighborhoods 

In [56]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 3, 1, 5, 1, 1, 1, 4, 1, 4])

In [57]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_analysis.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,2,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
51,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,4,Coffee Shop,Bakery,Italian Restaurant,Restaurant,Pub,Pizza Place,Café,Chinese Restaurant,Indian Restaurant,Pharmacy
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Café,Hotel,Mediterranean Restaurant,Yoga Studio,Men's Store
53,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,6,Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Theater,Cosmetics Shop,Shoe Store,Restaurant
54,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Pizza Place,Bookstore,Middle Eastern Restaurant,Lingerie Store


In [58]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

row_data =  zip(
    toronto_merged['Latitude'], 
    toronto_merged['Longitude'], 
    toronto_merged['Neighbourhood'], 
    toronto_merged['Cluster Labels']
)

for lat, lon, poi, cluster in row_data:
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(kclusters - 1, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,"University of Toronto, Harbord",Café,Restaurant,Bar,Japanese Restaurant,Bookstore,Sandwich Place,Bakery,Yoga Studio,Italian Restaurant,Beer Bar


In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(kclusters - 1, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Café,Hotel,Mediterranean Restaurant,Yoga Studio,Men's Store
57,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Japanese Restaurant,Burger Joint,Bubble Tea Shop,Department Store,Juice Bar
59,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Café,Hotel,Brewery,Scenic Lookout,Restaurant,Fried Chicken Joint,Baseball Stadium,Bank
60,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Seafood Restaurant,Salad Place,Japanese Restaurant,Italian Restaurant,Asian Restaurant
61,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Seafood Restaurant,Cocktail Bar,Bar,Bakery
70,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,Steakhouse,Asian Restaurant,Seafood Restaurant,American Restaurant


In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(kclusters - 1, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Rosedale,Park,Playground,Trail,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(kclusters - 1, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Bar,Rental Car Location,Sculpture Garden,Harbor / Marina,Boat or Ferry,Boutique


In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(kclusters - 1, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,"St. James Town, Cabbagetown",Coffee Shop,Bakery,Italian Restaurant,Restaurant,Pub,Pizza Place,Café,Chinese Restaurant,Indian Restaurant,Pharmacy
54,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Pizza Place,Bookstore,Middle Eastern Restaurant,Lingerie Store
55,St. James Town,Café,Coffee Shop,Cosmetics Shop,Clothing Store,Restaurant,American Restaurant,Cocktail Bar,Seafood Restaurant,Beer Bar,Hotel
56,Berczy Park,Coffee Shop,Farmers Market,Bakery,Café,Restaurant,Cocktail Bar,Seafood Restaurant,Beer Bar,Cheese Shop,Gym
58,"Richmond, Adelaide, King",Coffee Shop,Clothing Store,Café,Restaurant,Hotel,Gym,Bar,Thai Restaurant,Steakhouse,Office
67,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Vegetarian / Vegan Restaurant,Mexican Restaurant,Vietnamese Restaurant,Dessert Shop,Caribbean Restaurant,Bakery,Burger Joint,Pizza Place
69,Stn A PO Boxes,Coffee Shop,Café,Restaurant,Pub,Gym,Beer Bar,Seafood Restaurant,Japanese Restaurant,Hotel,Italian Restaurant


In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[2] + list(range(kclusters - 1, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,Christie,Grocery Store,Café,Park,Nightclub,Diner,Restaurant,Italian Restaurant,Baby Store,Candy Store,Athletics & Sports


In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[2] + list(range(kclusters - 1, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,"Regent Park, Harbourfront",Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Theater,Cosmetics Shop,Shoe Store,Restaurant
85,"Queen's Park, Ontario Provincial Government",Coffee Shop,College Cafeteria,Yoga Studio,Smoothie Shop,Beer Bar,Sandwich Place,Café,Portuguese Restaurant,Persian Restaurant,College Auditorium


# Conclusions

1. The toronto downtown it's is very similar.
2. The most common venue is coffee shop.
3. The cluster 0 is perfect for students.
4. The cluster 1 is perfect for workers.
5. The cluster 2 is ideal to have a fitness life.
6. The cluster 3 is basically the airport.
7. The cluster 4 is related to shopping.
3. The cluster 5 is good to people looking for a good place to live and the night life.
4. The cluster 6 is good to people looking for something quite in the downtown.